In [1]:
import numpy as np
import pandas as pd
import gc
import time
import pickle

from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import re


In [2]:
# Où sont les fichiers à charger
pathProject="/Users/anacoluthedev/Dev/Data/home-credit-default-risk/"

In [3]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [5]:
# Ouverture du modèle entrainé, en mode binaire
model_filename = pathProject+'lgb_model.pkl'
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [6]:
# Sauvegarde du dataframe, prétraité, pour son utilisation dans d'autres notebooks
df_filename = pathProject+'preprocessed_dataframe.csv'

df_API = pd.read_csv(df_filename)

NameError: name 'df_api' is not defined

In [8]:
df_API.sample(4)

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,INS_AMT_INSTALMENT_MEAN,INS_AMT_INSTALMENT_SUM,INS_AMT_PAYMENT_MIN,INS_AMT_PAYMENT_MAX,INS_AMT_PAYMENT_MEAN,INS_AMT_PAYMENT_SUM,INS_DAYS_ENTRY_PAYMENT_MAX,INS_DAYS_ENTRY_PAYMENT_MEAN,INS_DAYS_ENTRY_PAYMENT_SUM,INS_COUNT
299275,446707,0.0,0,1,0,0,202500.0,630000.0,32296.5,630000.0,...,5982.070500,59820.705,5970.285,5983.380,5982.070500,59820.705,-2644.0,-2776.900000,-27769.0,10.0
140021,262361,0.0,1,1,0,2,180000.0,810000.0,40500.0,810000.0,...,197551.873125,1580414.985,13659.435,1102614.525,197551.873125,1580414.985,-136.0,-303.250000,-2426.0,8.0
286512,431838,0.0,1,0,0,0,112500.0,207000.0,10066.5,207000.0,...,14125.637143,889915.140,0.135,135000.000,13867.613571,873659.655,-25.0,-968.523810,-61017.0,63.0
293660,440218,0.0,0,1,0,0,76500.0,592560.0,35937.0,450000.0,...,4619.380743,341834.175,4.590,8575.425,4092.042162,302811.120,-707.0,-1693.918919,-125350.0,74.0
